# Data Ingestion

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

train_raw_df = fetch_20newsgroups(subset='train', categories=categories)
test_raw_df = fetch_20newsgroups(subset='test', categories=categories)

x_train, x_val, y_train, y_val = train_test_split(np.array(train_raw_df.data), train_raw_df.target, test_size=0.1)
x_test = np.array(test_raw_df.data)
y_test = test_raw_df.target

# x_train = [x_train[:200] for x in x_train]

print('Train:', len(x_train))
print('Val:', len(x_val))
print('Test:', len(x_test))

Train: 2031
Val: 226
Test: 1502


In [2]:
%reload_ext  autoreload
%autoreload 2

import sys, os
def add_aion(curr_path=None):
    if curr_path is None:
        dir_path = os.getcwd()
        target_path = os.path.dirname(dir_path)
        if target_path not in sys.path:
            print('Added %s into sys.path.' % (target_path))
            sys.path.insert(0, target_path)
            
add_aion()

Added /data/jupyter/common into sys.path.


# Model (Keras)

In [3]:
max_sequence_length = 200

In [4]:
from aion.embeddings.cove import CoVeEmbeddings

cove_embs = CoVeEmbeddings(
    word_embeddings_dir='../model/text/stanford/glove/', 
    max_sequence_length=max_sequence_length, verbose=20)
tmp = cove_embs.load_model(dest_dir='../model/text/salesforce/cove/')

Using TensorFlow backend.


2018-10-06 16:04:42.665310. [FOUND] Keras_CoVe.h5 in ../model/text/salesforce/cove/


/anaconda/envs/py35/lib/python3.5/site-packages/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
x_train_t = cove_embs.encode(x_train)
x_test_t = cove_embs.encode(x_test)

In [6]:
x_train_t2 = x_train_t.reshape(len(x_train_t), max_sequence_length*600)
x_test_t2 = x_test_t.reshape(len(x_test_t), max_sequence_length*600)

In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='newton-cg')
model.fit(x_train_t2, y_train)

y_pred = model.predict(x_test_t2)

/anaconda/envs/py35/lib/python3.5/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


For sake of easier demonstration, I did not do any data preprocessing. It leads lots of OOV and causing the result bad.

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('Accuracy:%.2f%%' % (accuracy_score(y_test, y_pred)*100))
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy:32.56%
Classification Report:
             precision    recall  f1-score   support

          0       0.26      0.03      0.05       319
          1       0.34      0.61      0.43       389
          2       0.33      0.05      0.08       396
          3       0.32      0.56      0.41       398

avg / total       0.31      0.33      0.25      1502

